In [50]:
import pandas as pd

In [101]:
df2 = pd.read_csv('robocall-recipient-area-code.csv')

In [453]:
df = df2
print(len(df))

1439383


In [454]:
df = df[:1381807]
print(len(df))

1381807


In [455]:
df['Issue'].value_counts()

Unwanted Calls                                                      434839
Billing                                                             216496
Telemarketing (including do not call and spoofing)                  210768
Robocalls                                                           102594
Availability                                                         65431
Equipment                                                            57437
Open Internet/Net Neutrality                                         42572
Privacy                                                              40550
Interference                                                         37440
Availability (including rural call completion)                       33074
Speed                                                                29832
Indecency                                                            27781
Number Portability (keeping your number if you change providers)     20851
Junk Faxes               

In [456]:
#df = df.loc[df['Issue'] == 'Interference']
#df.head()

In [457]:
state_str = '''Alabama 	01 	AL
Alaska 	02 	AK
Arizona 	04 	AZ
Arkansas 	05 	AR
California 	06 	CA
Colorado 	08 	CO
Connecticut 	09 	CT
Delaware 	10 	DE
District of Columbia 	11 	DC
Florida 	12 	FL
Georgia 	13 	GA
Hawaii 	15 	HI
Idaho 	16 	ID
Illinois 	17 	IL
Indiana 	18 	IN
Iowa 	19 	IA
Kansas 	20 	KS
Kentucky 	21 	KY
Louisiana 	22 	LA
Maine 	23 	ME
Maryland 	24 	MD
Massachusetts 	25 	MA
Michigan 	26 	MI
Minnesota 	27 	MN
Mississippi 	28 	MS
Missouri 	29 	MO
Montana 	30 	MT
Nebraska 	31 	NE
Nevada 	32 	NV
New Hampshire 	33 	NH
New Jersey 	34 	NJ
New Mexico 	35 	NM
New York 	36 	NY
North Carolina 	37 	NC
North Dakota 	38 	ND
Ohio 	39 	OH
Oklahoma 	40 	OK
Oregon 	41 	OR
Pennsylvania 	42 	PA
Rhode Island 	44 	RI
South Carolina 	45 	SC
South Dakota 	46 	SD
Tennessee 	47 	TN
Texas 	48 	TX
Utah 	49 	UT
Vermont 	50 	VT
Virginia 	51 	VA
Washington 	53 	WA
West Virginia 	54 	WV
Wisconsin 	55 	WI
Wyoming 	56 	WY'''

In [458]:
states = state_str.split('\n')
states = pd.DataFrame(list(map(lambda x: x.split('\t'),states)),columns=['state','id','Short'])
df = pd.merge(left=df,right=states,left_on='State',right_on='Short',how='right')
df.head()

,Ticket ID,Ticket Created,Date of Issue,Time of Issue,Form,Method,Issue,Caller ID Number,Type of Call or Messge,Advertiser Business Number,City,State,Zip,Location,Recipient Location,Recipient Area Code,recipient_area_code_cat,state,id,Short
0,1000296,05/25/2016 11:15:29 AM +0000,05/01/2016,1:00 pm,Phone,Internet (VOIP),Robocalls,866-410-0458,Autodialed Live Voice Call,NaN,Plantation,FL,33324,"FL 33324\n(26.11294, -80.27429)","(26.008045, -80.248042)",754.0,0,Florida,12,FL
1,1000342,05/25/2016 01:32:27 PM +0000,05/24/2016,5:59 P.M.,Phone,Wireless (cell phone/other mobile device),Telemarketing (including do not call and spoof...,281-808-2308,Live Voice,NaN,Tampa,FL,33605,"FL 33605\n(27.96128, -82.429811)","(27.96128, -82.429811)",813.0,10,Florida,12,FL
2,1000374,05/25/2016 02:04:17 PM +0000,05/23/2016,3:01 pm,Phone,Wired,Telemarketing (including do not call and spoof...,941-467-3194,Live Voice,NaN,Port Charlotte,FL,33952,"FL 33952\n(26.992249, -82.096316)","(27.000998, -82.12316)",941.0,14,Florida,12,FL
3,1000452,05/25/2016 02:53:50 PM +0000,05/23/2016,6:00 pm,Phone,Wired,Telemarketing (including do not call and spoof...,305-823-2345,Live Voice,NaN,Pembroke Pines,FL,33029,"FL 33029\n(25.998658, -80.400741)","(25.998658, -80.400741)",754.0,0,Florida,12,FL
4,1000466,05/25/2016 03:12:15 PM +0000,05/25/2016,10:00 am,Phone,Wireless (cell phone/other mobile device),Telemarketing (including do not call and spoof...,NaN,Abandoned Calls,NaN,Lake Worth,FL,33462,"FL 33462\n(26.57989, -80.076308)","(26.575863, -80.085958)",561.0,30,Florida,12,FL


In [459]:
df = df.drop(['State'], axis=1)

In [460]:
print(len(df))

1378306


In [461]:
import re
def split_coordinates(df):
    df=df.rename(columns = {'Ticket ID':'FMID'})
    df=df.rename(columns = {'Zip':'MarketName'})
    df=df.rename(columns = {'City':'city'})
    df=df.rename(columns = {'Recipient Area Code':'County'})
    df=df.rename(columns = {'state':'State'})
    
    lat = []
    lng = []
    for i in range(len(df)):
        loc = df.iloc[i]['Recipient Location']
        coor = re.findall(r'[+-]?\d+(?:\.\d+)?', loc)
        lat.append(float(coor[0]))
        lng.append(float(coor[1]))
    df['lat'] = lat
    df['lng'] = lng
    df=df.rename(columns = {'Method':'Website'})
    df.drop(['Ticket Created', 'Date of Issue'], axis=1)
    
    return df

    
        
        
        

In [462]:
dfs = split_coordinates(df)
dfs.head()

,FMID,Ticket Created,Date of Issue,Time of Issue,Form,Website,Issue,Caller ID Number,Type of Call or Messge,Advertiser Business Number,...,MarketName,Location,Recipient Location,County,recipient_area_code_cat,State,id,Short,lat,lng
0,1000296,05/25/2016 11:15:29 AM +0000,05/01/2016,1:00 pm,Phone,Internet (VOIP),Robocalls,866-410-0458,Autodialed Live Voice Call,NaN,...,33324,"FL 33324\n(26.11294, -80.27429)","(26.008045, -80.248042)",754.0,0,Florida,12,FL,26.008045,-80.248042
1,1000342,05/25/2016 01:32:27 PM +0000,05/24/2016,5:59 P.M.,Phone,Wireless (cell phone/other mobile device),Telemarketing (including do not call and spoof...,281-808-2308,Live Voice,NaN,...,33605,"FL 33605\n(27.96128, -82.429811)","(27.96128, -82.429811)",813.0,10,Florida,12,FL,27.961280,-82.429811
2,1000374,05/25/2016 02:04:17 PM +0000,05/23/2016,3:01 pm,Phone,Wired,Telemarketing (including do not call and spoof...,941-467-3194,Live Voice,NaN,...,33952,"FL 33952\n(26.992249, -82.096316)","(27.000998, -82.12316)",941.0,14,Florida,12,FL,27.000998,-82.123160
3,1000452,05/25/2016 02:53:50 PM +0000,05/23/2016,6:00 pm,Phone,Wired,Telemarketing (including do not call and spoof...,305-823-2345,Live Voice,NaN,...,33029,"FL 33029\n(25.998658, -80.400741)","(25.998658, -80.400741)",754.0,0,Florida,12,FL,25.998658,-80.400741
4,1000466,05/25/2016 03:12:15 PM +0000,05/25/2016,10:00 am,Phone,Wireless (cell phone/other mobile device),Telemarketing (including do not call and spoof...,NaN,Abandoned Calls,NaN,...,33462,"FL 33462\n(26.57989, -80.076308)","(26.575863, -80.085958)",561.0,30,Florida,12,FL,26.575863,-80.085958


In [463]:
dfs = dfs.drop([
 'Ticket Created',
 'Date of Issue',
 'Time of Issue',
 'Form',
 'Issue',
 'Caller ID Number',
 'Advertiser Business Number',
 'Location',
 'Recipient Location',
 'recipient_area_code_cat', 'id', 'Short'], axis=1)

dfs.head()

,FMID,Website,Type of Call or Messge,city,MarketName,County,State,lat,lng
0,1000296,Internet (VOIP),Autodialed Live Voice Call,Plantation,33324,754.0,Florida,26.008045,-80.248042
1,1000342,Wireless (cell phone/other mobile device),Live Voice,Tampa,33605,813.0,Florida,27.961280,-82.429811
2,1000374,Wired,Live Voice,Port Charlotte,33952,941.0,Florida,27.000998,-82.123160
3,1000452,Wired,Live Voice,Pembroke Pines,33029,754.0,Florida,25.998658,-80.400741
4,1000466,Wireless (cell phone/other mobile device),Abandoned Calls,Lake Worth,33462,561.0,Florida,26.575863,-80.085958


In [464]:
list(dfs)

['FMID',
 'Website',
 'Type of Call or Messge',
 'city',
 'MarketName',
 'County',
 'State',
 'lat',
 'lng']

In [465]:
dfs = dfs[['FMID', 'MarketName', 'city', 'County', 'State', 'lat', 'lng', 'Website']]
dfs.head()

,FMID,MarketName,city,County,State,lat,lng,Website
0,1000296,33324,Plantation,754.0,Florida,26.008045,-80.248042,Internet (VOIP)
1,1000342,33605,Tampa,813.0,Florida,27.961280,-82.429811,Wireless (cell phone/other mobile device)
2,1000374,33952,Port Charlotte,941.0,Florida,27.000998,-82.123160,Wired
3,1000452,33029,Pembroke Pines,754.0,Florida,25.998658,-80.400741,Wired
4,1000466,33462,Lake Worth,561.0,Florida,26.575863,-80.085958,Wireless (cell phone/other mobile device)


In [466]:
print(len(dfs))

1378306


In [467]:
dfs.to_json('data/Overall.json', orient = 'records')